<a href="https://colab.research.google.com/github/e40125/118A-Final-CNM06/blob/main/caruana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy pandas scikit-learn

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import datasets as skd
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

# Evaluation metric
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [ ]:
# Data cleaning for HRAnalytics data set from Kaggle
# Making dictionaries of ordinal features

gender_map = {
    'Female': 2,
    'Male': 1,
    'Other': 0
}

relevent_experience_map = {
    'Has relevent experience': 1,
    'No relevent experience': 0
}

enrolled_university_map = {
    'no_enrollment': 0,
    'Full time course': 1,
    'Part time course': 2
}

education_level_map = {
    'Primary School': 0,
    'Graduate': 2,
    'Masters': 3,
    'High School': 1,
    'Phd': 4
}

major_map = {
    'STEM': 0,
    'Business Degree': 1,
    'Arts': 2,
    'Humanities': 3,
    'No Major': 4,
    'Other': 5
}

experience_map = {
    '<1': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '10': 10,
    '11': 11,
    '12': 12,
    '13': 13,
    '14': 14,
    '15': 15,
    '16': 16,
    '17': 17,
    '18': 18,
    '19': 19,
    '20': 20,
    '>20': 21
}

company_type_map = {
    'Pvt Ltd': 0,
    'Funded Startup': 1,
    'Early Stage Startup': 2,
    'Other': 3,
    'Public Sector': 4,
    'NGO': 5
}

company_size_map = {
    '<10': 0,
    '10/49': 1,
    '100-500': 2,
    '1000-4999': 3,
    '10000+': 4,
    '50-99': 5,
    '500-999': 6,
    '5000-9999': 7
}

last_new_job_map = {
    'never': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '>4': 5
}
# preparing data, calculate positive rate
def read_letterP1_dataset():
    print()
    print('Preparing letter.P1 dataset...')

    df = pd.read_csv("letter-recognition.csv")

    # if O then 1
    data = df.copy()
    data['letter'] = data['letter'].apply(lambda x: 1 if x == 'O' else 0)
    # pos rate
    O = len(data[data['letter'] == 1])
    non_O = len(data[data['letter'] == 0])
    poz = O / len(data)

    print('O / non-O= {} / {}'.format(O, non_O))
    print('POSITIVE SAMPLE = {}'.format(poz*100))
    print('# features = {}'.format(len(data.columns) - 1))

    labels = data['letter']
    del data['letter']
    features = data.copy()

    return features, labels

def read_letterP2_dataset():
    print()
    print('Preparing letter.P2 dataset...')

    df = pd.read_csv("letter-recognition.csv")

    data = df.copy()
    data['letter'] = data['letter'].apply(lambda x: 1 if x <= 'M' else 0)

    A_M = len(data[data['letter'] == 1])
    N_Z = len(data[data['letter'] == 0])
    poz = A_M / len(data)

    print('A-M / N-Z = {} / {}'.format(A_M, N_Z))
    print('POSITIVE SAMPLE = {}'.format(poz*100))
    print('# features = {}'.format(len(data.columns) - 1))

    labels = data['letter']
    del data['letter']
    features = data.copy()

    return features, labels

def read_adult_dataset():
    print()
    print('Preparing ADULT dataset...')

    df = pd.read_csv("adult.csv")

    data = df.copy()
    # replace ? with most frquent
    attrib, counts = np.unique(data['workclass'], return_counts = True)
    most_freq_attrib = attrib[np.argmax(counts, axis = 0)]
    data.loc[data['workclass'] == '?', 'workclass'] = most_freq_attrib

    attrib, counts = np.unique(data['occupation'], return_counts = True)
    most_freq_attrib = attrib[np.argmax(counts, axis = 0)]
    data.loc[data['occupation'] == '?', 'occupation'] = most_freq_attrib
    # US disproportional so drop other countries
    attrib, counts = np.unique(data['native-country'], return_counts = True)
    most_freq_attrib = attrib[np.argmax(counts, axis = 0)]
    data.loc[data['native-country'] == '?', 'native-country'] = most_freq_attrib
    data['native-country'] = data['native-country'].apply(lambda x: 1 if x.strip() == "United-States" else 0)

    data['income'] = data['income'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})

    # Education and Education-num are the same thing, fnlwgt dont help
    data.drop(['fnlwgt', 'education'], axis=1, inplace=True)

    data = pd.get_dummies(data, columns=['workclass', 'occupation', 'relationship',
                                         'race', 'gender', 'marital-status', 'native-country'])

    over_50 = len(data[data['income'] == 1])
    less_50 = len(data[data['income'] == 0])
    poz = over_50 / len(data)

    print('>50K / <=50K = {} / {}'.format(over_50, less_50))
    print('POSITIVE SAMPLE = {}'.format(poz*100))
    print('# features = {}'.format(len(data.columns) - 1))

    labels = data['income']
    del data['income']
    features = data.copy()

    return features, labels

def read_HRAnalytics_dataset():
    print()
    print('Preparing HRAnalytics dataset...')

    df = pd.read_csv("hr_analytics.csv")

    # Transforming Categorical features into numarical features
    df.loc[:,'education_level'] = df['education_level'].map(education_level_map)
    df.loc[:,'company_size'] = df['company_size'].map(company_size_map)
    df.loc[:,'company_type'] = df['company_type'].map(company_type_map)
    df.loc[:,'last_new_job'] = df['last_new_job'].map(last_new_job_map)
    df.loc[:,'major_discipline'] = df['major_discipline'].map(major_map)
    df.loc[:,'enrolled_university'] = df['enrolled_university'].map(enrolled_university_map)
    df.loc[:,'relevent_experience'] = df['relevent_experience'].map(relevent_experience_map)
    df.loc[:,'gender'] = df['gender'].map(gender_map)
    df.loc[:,'experience'] = df['experience'].map(experience_map)

    # encoding city feature using label encoder, alternative to one-hot
    lb_en = LabelEncoder()
    df.loc[:,'city'] = lb_en.fit_transform(df.loc[:,'city'])

    # Deal with missing values
    missing_cols = df.columns[df.isna().any()].tolist()

    # dataframe having features with missing values
    df_missing = df[['enrollee_id'] + missing_cols]
    # dataframe having features without missing values
    df_non_missing = df.drop(missing_cols, axis = 1)

    #k-Nearest Neighbour Imputation
    knn_imputer = KNNImputer(n_neighbors = 3)

    #Rounding them because these are categorical features
    X = np.round(knn_imputer.fit_transform(df_missing))

    df_missing = pd.DataFrame(X, columns = df_missing.columns)

    df = pd.merge(df_missing, df_non_missing, on = 'enrollee_id')

    y = len(df[df['target'] == 1])
    n = len(df[df['target'] == 0])
    poz = y / len(df)

    print('y / n = {} / {}'.format(y, n))
    print('POSITIVE SAMPLE = {}'.format(poz*100))
    print('# features = {}'.format(len(df.columns) - 2))

    features = df.drop(['enrollee_id', 'target'], axis=1)
    labels = df['target']

    return features, labels
# goal of scaling change to [0,1], so larger numbers don't dominate
def scale_columns_legit(df, column_names, scaler):
    d = df.copy()
    for column_name in column_names:
        if (not column_name in d.columns):
            print(column_name)
            return

    scaled_array = scaler.transform(d[column_names])
    scaled_df = pd.DataFrame(scaled_array, columns=column_names)
    scaled_df.index = d.index

    d.drop(columns=column_names, inplace=True)

    return pd.concat([d, scaled_df], axis=1)

def letter_dataset_scaling(X_train, X_test):
    #let scaler only know about training set
    cols_to_scale_standard = X_train.columns

    # Calculate the standard varibles of training set
    standardSc = StandardScaler()
    standardSc.fit(X_train[cols_to_scale_standard])

    # Transform using the variables calculated above over "X_train" and "X_test"
    X_train = scale_columns_legit(X_train, cols_to_scale_standard, standardSc)
    X_test = scale_columns_legit(X_test, cols_to_scale_standard, standardSc)

    return X_train, X_test
def HRAnalytics_dataset_scaling(X_train, X_test):
    # No scaling
    return X_train, X_test

def adult_dataset_scaling(X_train, X_test):
    def scale_columns_legit(df, column_names, scaler):
        d = df.copy()
        for column_name in column_names:
            if (not column_name in d.columns):
                print(column_name)
                return

        scaled_array = scaler.transform(d[column_names])
        scaled_df = pd.DataFrame(scaled_array, columns=column_names)
        scaled_df.index = d.index

        d.drop(columns=column_names, inplace=True)

        return pd.concat([d, scaled_df], axis=1)

    #let scaler only know about training set
    cols_to_scale_standard = ['hours-per-week', 'age', 'educational-num']
    cols_to_scale_robust = ['capital-gain', 'capital-loss']

    standardSc = StandardScaler()
    standardSc.fit(X_train[cols_to_scale_standard])

    # income
    robustSc = RobustScaler()
    robustSc.fit(X_train[cols_to_scale_robust])

    # Transform X_train and X_test
    X_train = scale_columns_legit(X_train, cols_to_scale_standard, standardSc)
    X_train = scale_columns_legit(X_train, cols_to_scale_robust, robustSc)
    # standscaler is from training data then applied to test data
    X_test = scale_columns_legit(X_test, cols_to_scale_standard, standardSc)
    X_test = scale_columns_legit(X_test, cols_to_scale_robust, robustSc)

    return X_train, X_test

# the data sets
datasets = {'letterP1': read_letterP1_dataset(),
            'letterP2': read_letterP2_dataset(),
            'adult': read_adult_dataset(),
            'hr': read_HRAnalytics_dataset()}

scaling_fns = {'letterP1': letter_dataset_scaling,
               'letterP2': letter_dataset_scaling,
               'adult': adult_dataset_scaling,
               'hr': HRAnalytics_dataset_scaling}


Preparing letter.P1 dataset...
O / non-O= 753 / 19247
POSITIVE SAMPLE = 3.765
# features = 16

Preparing letter.P2 dataset...
A-M / N-Z = 9940 / 10060
POSITIVE SAMPLE = 49.7
# features = 16

Preparing ADULT dataset...
>50K / <=50K = 11687 / 37155
POSITIVE SAMPLE = 23.928176569346054
# features = 49

Preparing HRAnalytics dataset...
y / n = 4777 / 14381
POSITIVE SAMPLE = 24.934753105752165
# features = 12


In [ ]:
# the models
models = {
    'MLPClassifier': MLPClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'KNeighboursClassifier': KNeighborsClassifier(),
#   'SVM': SVC(),
#   'DecisionTreeClassifier': DecisionTreeClassifier()
}

# the optimization parameters for each of the above models
params = {
    'MLPClassifier': {'max_iter': [500, 1000],
                      'hidden_layer_sizes': [[32], [64],[128]],
                      'solver':["lbfgs", "sgd", "adam"]
    },
    'RandomForestClassifier':{
        'n_estimators': [125, 250, 500, 1000],
        'max_features': ['sqrt', 'log2']
    },
    'KNeighboursClassifier': {
        'n_neighbors': np.arange(3,31,3),
        'weights': ['uniform', 'distance'],
        'algorithm': ['ball_tree', 'kd_tree', 'brute']
    },
    'GradientBoostingClassifier': {
        "max_features":["log2", "sqrt"],
        "n_estimators":[10, 50, 100, 500, 1000, 2000]
    },
}

In [11]:
cv_results = {}
test_results = {}
for scaling in [True, False]:
    for dataset_name in datasets:
        print('Evaluating {} dataset...'.format(dataset_name))

        features, labels = datasets[dataset_name]
        scaling_fn = scaling_fns[dataset_name]

        X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size=5000, random_state=42)

        if scaling:
            if dataset_name == 'hr':
                continue

            X_train, X_test = scaling_fn(X_train, X_test)

        print('Number of Training Sample: {}'.format(len(X_train)))
        print('Number of Testing Sample: {}'.format(len(X_test)))

        for model_name in models:
            est = models[model_name]
            est_params = params[model_name]

            if scaling:
                dict_key = '{}_{}__SCALE'.format(dataset_name, model_name)
            else:
                dict_key = '{}_{}__NOSCALE'.format(dataset_name, model_name)

            print('Testing {} model...'.format(est))

            # Grid Search with 5-fold cross validation
            gscv = GridSearchCV(estimator=est, param_grid=est_params, cv=5, verbose=3, n_jobs=-1)
            gscv.fit(X_train, y_train)
            print("Best parameters: {}".format(gscv.best_params_))
            print(gscv.best_score_)

            # Fit on the whole training set
            best_est = gscv.best_estimator_
            best_est.fit(X_train, y_train)
            # Make predictions on test set
            y_test_pred = best_est.predict(X_test)

            # Get pred scores
            print()
            print('--------------------------')
            print('Evaluating test set... (Scaling = {})'.format(scaling))

            accuracy = accuracy_score(y_test, y_test_pred)
            f1 = f1_score(y_test, y_test_pred)
            auc = roc_auc_score(y_test, y_test_pred)

            cv_results[dict_key] = pd.DataFrame(gscv.cv_results_)
            test_results[dict_key] = [accuracy, f1, auc]

            print('* Accuracy: {}'.format(accuracy))
            print('* F1-score: {}'.format(f1))
            print('* AUC:      {}'.format(auc))

            print('--------------------------')
            print()

Evaluating letterP1 dataset...
Number of Training Sample: 5000
Number of Testing Sample: 15000
Testing MLPClassifier() model...
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best parameters: {'hidden_layer_sizes': [64], 'max_iter': 500, 'solver': 'adam'}
0.9936

--------------------------
Evaluating test set... (Scaling = True)
* Accuracy: 0.9915333333333334
* F1-score: 0.8846503178928247
* AUC:      0.9329516224772458
--------------------------

Testing GradientBoostingClassifier() model...
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters: {'max_features': 'sqrt', 'n_estimators': 2000}
0.9917999999999999

--------------------------
Evaluating test set... (Scaling = True)
* Accuracy: 0.9919333333333333
* F1-score: 0.8882733148661126
* AUC:      0.9280099920854767
--------------------------

Testing RandomForestClassifier() model...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters: {'max_features': 'log2', 'n_estimators': 250}
0.9875999999999999

--------------------------
Evaluating test set... (Scaling = True)
* Accuracy: 0.9884666666666667
* F1-score: 0.8247213779128673
* AUC:      0.8627003363672338
--------------------------

Testing KNeighborsClassifier() model...
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters: {'algorithm': 'ball_tree', 'n_neighbors': 6, 'weights': 'distance'}
0.9911999999999999

--------------------------
Evaluating test set... (Scaling = True)
* Accuracy: 0.9902
* F1-score: 0.8695652173913043
* AUC:      0.9348337950138503


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Best parameters: {'hidden_layer_sizes': [128], 'max_iter': 1000, 'solver': 'lbfgs'}
0.9148


...


--------------------------
Evaluating test set... (Scaling = False)
* Accuracy: 0.9260666666666667
* F1-score: 0.9251838359306483
* AUC:      0.9260351550544185
--------------------------

Testing GradientBoostingClassifier() model...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters: {'max_features': 'log2', 'n_estimators': 2000}
0.933

--------------------------
Evaluating test set... (Scaling = False)
* Accuracy: 0.9426666666666667
* F1-score: 0.942305112035422
* AUC:      0.9426859149701742
--------------------------

Testing RandomForestClassifier() model...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters: {'max_features': 'log2', 'n_estimators': 500}
0.9389999999999998

--------------------------
Evaluating test set... (Scaling = False)
* Accuracy: 0.9512
* F1-score: 0.9508064516129032
* AUC:      0.9512044436755139
--------------------------

Testing KNeighborsClassifier() model...
Fitting 5 folds for each of 60 candidates,

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Best parameters: {'hidden_layer_sizes': [64], 'max_iter': 1000, 'solver': 'lbfgs'}
0.7668


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)



--------------------------
Evaluating test set... (Scaling = False)
* Accuracy: 0.7683288600084758
* F1-score: 0.42837225514116417
* AUC:      0.6280151520348198
--------------------------

Testing GradientBoostingClassifier() model...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters: {'max_features': 'log2', 'n_estimators': 50}
0.7828

--------------------------
Evaluating test set... (Scaling = False)
* Accuracy: 0.7797005226726939
* F1-score: 0.4639972503866644
* AUC:      0.6470311071037136
--------------------------

Testing RandomForestClassifier() model...
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best parameters: {'max_features': 'log2', 'n_estimators': 250}
0.7756000000000001

--------------------------
Evaluating test set... (Scaling = False)
* Accuracy: 0.7709422234778923
* F1-score: 0.4374674761491761
* AUC:      0.6327816961210654
--------------------------

Testing KNeighborsClassifier() model...
Fitting 5 folds for each of 6

In [12]:
df_results = pd.DataFrame.from_dict(test_results).T
df_results.columns = ['Accuracy', 'F1', 'AUC']

# Pre-processing and generate table

_df = df_results.copy()

info = _df.index.values
info = [x.split('_') for x in info]

# Combine the second and third elements (if the third is empty)
info = [row[:2] + ['_'.join(row[2:])] for row in info]

# Create a list from the zip object
info_list = list(zip(*info))

# Unpack the info_list into new columns
_df['Dataset'], _df['Method'], _df['Scaling'] = info_list

# Reorder columns
_df = _df[['Dataset', 'Method', 'Scaling', 'Accuracy', 'F1', 'AUC']]

# Sort values by Dataset
_df = _df.sort_values(by=['Dataset'])

# Calculate Average score
_df['Average score'] = _df.apply(lambda row: (row['Accuracy'] + row['F1'] + row['AUC']) / 3, axis=1)

# Reset index
_df = _df.reset_index(drop=True)

print(_df.head(36))

     Dataset                      Method   Scaling  Accuracy        F1  \
0      adult       KNeighboursClassifier  _NOSCALE  0.844647  0.633601   
1      adult      RandomForestClassifier  _NOSCALE  0.845422  0.654464   
2      adult  GradientBoostingClassifier  _NOSCALE  0.863761  0.696478   
3      adult               MLPClassifier  _NOSCALE  0.848000  0.655998   
4      adult               MLPClassifier    _SCALE  0.829570  0.622664   
5      adult  GradientBoostingClassifier    _SCALE  0.867410  0.701545   
6      adult      RandomForestClassifier    _SCALE  0.845605  0.654272   
7      adult       KNeighboursClassifier    _SCALE  0.858857  0.689482   
8         hr       KNeighboursClassifier  _NOSCALE  0.760842  0.344307   
9         hr  GradientBoostingClassifier  _NOSCALE  0.779701  0.463997   
10        hr               MLPClassifier  _NOSCALE  0.768329  0.428372   
11        hr      RandomForestClassifier  _NOSCALE  0.770942  0.437467   
12  letterP1       KNeighboursClassifi

In [13]:
# For calculating table values

In [14]:
_df = _df.sort_values(['Method','Scaling'])
__df = _df.copy()

In [15]:
_df.sort_values('Accuracy')

,Dataset,Method,Scaling,Accuracy,F1,AUC,Average score
8,hr,KNeighboursClassifier,_NOSCALE,0.760842,0.344307,0.590880,0.565343
10,hr,MLPClassifier,_NOSCALE,0.768329,0.428372,0.628015,0.608239
11,hr,RandomForestClassifier,_NOSCALE,0.770942,0.437467,0.632782,0.613730
9,hr,GradientBoostingClassifier,_NOSCALE,0.779701,0.463997,0.647031,0.630243
4,adult,MLPClassifier,_SCALE,0.829570,0.622664,0.747432,0.733222
0,adult,KNeighboursClassifier,_NOSCALE,0.844647,0.633601,0.748236,0.742161
1,adult,RandomForestClassifier,_NOSCALE,0.845422,0.654464,0.766167,0.755351
6,adult,RandomForestClassifier,_SCALE,0.845605,0.654272,0.765858,0.755245
3,adult,MLPClassifier,_NOSCALE,0.848000,0.655998,0.765751,0.756583
7,adult,KNeighboursClassifier,_SCALE,0.858857,0.689482,0.789869,0.779403


In [16]:
__df.sort_values('Average score')

,Dataset,Method,Scaling,Accuracy,F1,AUC,Average score
8,hr,KNeighboursClassifier,_NOSCALE,0.760842,0.344307,0.590880,0.565343
10,hr,MLPClassifier,_NOSCALE,0.768329,0.428372,0.628015,0.608239
11,hr,RandomForestClassifier,_NOSCALE,0.770942,0.437467,0.632782,0.613730
9,hr,GradientBoostingClassifier,_NOSCALE,0.779701,0.463997,0.647031,0.630243
4,adult,MLPClassifier,_SCALE,0.829570,0.622664,0.747432,0.733222
0,adult,KNeighboursClassifier,_NOSCALE,0.844647,0.633601,0.748236,0.742161
6,adult,RandomForestClassifier,_SCALE,0.845605,0.654272,0.765858,0.755245
1,adult,RandomForestClassifier,_NOSCALE,0.845422,0.654464,0.766167,0.755351
3,adult,MLPClassifier,_NOSCALE,0.848000,0.655998,0.765751,0.756583
7,adult,KNeighboursClassifier,_SCALE,0.858857,0.689482,0.789869,0.779403


In [17]:
gb = __df[__df['Dataset']=='letterP1']
gb

,Dataset,Method,Scaling,Accuracy,F1,AUC,Average score
19,letterP1,GradientBoostingClassifier,_NOSCALE,0.991867,0.886827,0.925401,0.934698
17,letterP1,GradientBoostingClassifier,_SCALE,0.991933,0.888273,0.928010,0.936072
12,letterP1,KNeighboursClassifier,_NOSCALE,0.990267,0.869643,0.932294,0.930734
15,letterP1,KNeighboursClassifier,_SCALE,0.990200,0.869565,0.934834,0.931533
18,letterP1,MLPClassifier,_NOSCALE,0.986867,0.828844,0.921945,0.912552
14,letterP1,MLPClassifier,_SCALE,0.991533,0.884650,0.932952,0.936378
13,letterP1,RandomForestClassifier,_NOSCALE,0.988000,0.816701,0.857309,0.887336
16,letterP1,RandomForestClassifier,_SCALE,0.988467,0.824721,0.862700,0.891963


In [18]:
_df.sort_values(['Method','Scaling'])

,Dataset,Method,Scaling,Accuracy,F1,AUC,Average score
2,adult,GradientBoostingClassifier,_NOSCALE,0.863761,0.696478,0.792527,0.784256
9,hr,GradientBoostingClassifier,_NOSCALE,0.779701,0.463997,0.647031,0.630243
19,letterP1,GradientBoostingClassifier,_NOSCALE,0.991867,0.886827,0.925401,0.934698
22,letterP2,GradientBoostingClassifier,_NOSCALE,0.942667,0.942305,0.942686,0.942553
5,adult,GradientBoostingClassifier,_SCALE,0.867410,0.701545,0.794231,0.787729
17,letterP1,GradientBoostingClassifier,_SCALE,0.991933,0.888273,0.928010,0.936072
26,letterP2,GradientBoostingClassifier,_SCALE,0.940800,0.940483,0.940827,0.940703
0,adult,KNeighboursClassifier,_NOSCALE,0.844647,0.633601,0.748236,0.742161
8,hr,KNeighboursClassifier,_NOSCALE,0.760842,0.344307,0.590880,0.565343
12,letterP1,KNeighboursClassifier,_NOSCALE,0.990267,0.869643,0.932294,0.930734


In [22]:
all_results = pd.concat([pd.DataFrame(v).assign(model=k) for k, v in cv_results.items()])
print(all_results)


letterP1_MLPClassifier__SCALE:
    mean_test_score  std_test_score  rank_test_score
0            0.9922          0.0023                6
1            0.9616          0.0005               17
2            0.9922          0.0013                6
3            0.9924          0.0022                5
4            0.9616          0.0008               17
5            0.9922          0.0007                6
6            0.9916          0.0015                9
7            0.9622          0.0015               14
8            0.9936          0.0010                1
9            0.9914          0.0005               12
10           0.9620          0.0013               15
11           0.9934          0.0019                2
12           0.9916          0.0008                9
13           0.9626          0.0005               13
14           0.9930          0.0013                3
15           0.9916          0.0024                9
16           0.9618          0.0007               16
17           0